In [3]:
import requests
import pandas as pd

# --- 1. Récupération des résultats de courses terminées (historique) ---

url_results = "https://api.jolpi.ca/ergast/f1/2025/results/"
response = requests.get(url_results)
data = response.json()
races = data['MRData']['RaceTable']['Races']

results = []
for race in races:
    for result in race['Results']:
        if result['status'] == 'Finished':
            results.append({
                'Round': int(race['round']),
                'Race': race['raceName'],
                'Date': race['date'],
                'Driver': result['Driver']['familyName'],
                'Constructor': result['Constructor']['name'],
                'Grid': int(result['grid']),
                'Position': int(result['position']),
                'Status': result['status'],
                'Points': float(result['points'])
            })

df_history = pd.DataFrame(results)

# Statistiques globales des pilotes
points = df_history.groupby('Driver')['Points'].sum()
victoires = df_history[df_history['Position'] == 1].groupby('Driver').size()
position_moy = df_history.groupby('Driver')['Position'].mean()

df_history_stats = pd.DataFrame({
    'Points': points,
    'Victoires': victoires,
    'Position_moy': position_moy
}).fillna(0)

practice_dfs = []

for session in [1, 2]:
    practice_url = f"https://api.jolpi.ca/ergast/f1/2025/8/practice/{session}/"
    practice_response = requests.get(practice_url)

    # Affiche le statut et le contenu brut pour vérifier
    print(f"Status: {practice_response.status_code}")
    print("Contenu brut de la réponse :")
    print(practice_response.text)  # <- ici tu verras si c'est du HTML, vide, etc.
    
    practice_data = practice_response.json()
    race_data = practice_data['MRData']['RaceTable']['Races']

    if race_data:
        practice_results = race_data[0].get('PracticeResults', [])
        for p in practice_results:
            if 'position' in p and p['position'].isdigit():
                practice_dfs.append({
                    'Driver': p['Driver']['familyName'],
                    'Practice': session,
                    'PracticePosition': int(p['position'])
                })

df_all_practices = pd.DataFrame(practice_dfs)

# Calcul de la position moyenne en essais par pilote
df_practice_avg = df_all_practices.groupby('Driver')['PracticePosition'].mean().reset_index()
df_practice_avg.rename(columns={'PracticePosition': 'AvgPracticePosition'}, inplace=True)

# --- 4. Fusion avec historique et calcul du score final ---

df = df_history_stats.merge(df_practice_avg, on='Driver', how='inner')
df['InvAvgPracticePos'] = 1 / df['AvgPracticePosition']
df['InvPosMoy'] = 1 / df['Position_moy']

# Normalisation
df['Points_norm'] = df['Points'] / df['Points'].max()
df['Victoires_norm'] = df['Victoires'] / df['Victoires'].max()
df['InvPosMoy_norm'] = df['InvPosMoy'] / df['InvPosMoy'].max()
df['InvAvgPracticePos_norm'] = df['InvAvgPracticePos'] / df['InvAvgPracticePos'].max()

# Score final (pondération ajustable)
df['PronosticScore'] = (
    df['Points_norm'] * 0.4 +
    df['Victoires_norm'] * 0.15 +
    df['InvPosMoy_norm'] * 0.2 +
    df['InvAvgPracticePos_norm'] * 0.25
)

# Classement final
df = df.sort_values(by='PronosticScore', ascending=False)
print(df[['Driver', 'PronosticScore']])

Status: 404
Contenu brut de la réponse :

<!doctype html>
<html lang="en">
<head>
  <title>Not Found</title>
</head>
<body>
  <h1>Not Found</h1><p>The requested resource was not found on this server.</p>
</body>
</html>



JSONDecodeError: Expecting value: line 2 column 1 (char 1)